ModSim Project 1

Arwen Sadler and Robin Graham-Hayes

In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

from modsim import *
# import everything from SymPy.
from sympy import *

#Import read_csv function from pandas
from pandas import read_csv

Q: How will marine mammal populations in the Alaskan area change in the next 100 years if current trends hold?

M:

-Put data into tables

-Graph data

-Build a model that fits the curve of the graph

-Use model to project populations into the future

If time:

-Add more species?

-Use data from climate change--CO2 levels or temperature rise, and use that to predict population change instead of just current trends

-Use data from marine mammal hunting trends--stochastic, where chance of getting killed per year is random?

In [75]:
#Read the csv files that contain data for every year
data2016 = read_csv('2016.csv', encoding = "ISO-8859-1")
data2015 = read_csv('2015.csv', encoding = "ISO-8859-1")
data2014 = read_csv('2014.csv', encoding = "ISO-8859-1")
data2013 = read_csv('2013.csv', encoding = "ISO-8859-1")
data2012 = read_csv('2012.csv', encoding = "ISO-8859-1")
data2011 = read_csv('2011.csv', encoding = "ISO-8859-1")
data2010 = read_csv('2010.csv', encoding = "ISO-8859-1")
data2009 = read_csv('2009.csv', encoding = "ISO-8859-1")
data2008 = read_csv('2008.csv', encoding = "ISO-8859-1")
data2007 = read_csv('2007.csv', encoding = "ISO-8859-1")
data2006 = read_csv('2006.csv', encoding = "ISO-8859-1")
data2005 = read_csv('2005.csv', encoding = "ISO-8859-1")
data2003 = read_csv('2003.csv', encoding = "ISO-8859-1")
data2001 = read_csv('2001.csv', encoding = "ISO-8859-1")
data2000 = read_csv('2000.csv', encoding = "ISO-8859-1")
data1999 = read_csv('1999.csv', encoding = "ISO-8859-1")
data1998 = read_csv('1998.csv', encoding = "ISO-8859-1")
data1996 = read_csv('1996-clean.csv', encoding = "ISO-8859-1");
data1995 = read_csv('1995-clean.csv', encoding = "ISO-8859-1")

,Species,Stock,population
0,Baird's Beaked Whale,Alaska,-1
1,Bearded Seal,Alaska,-1
2,Beluga Whale,Beaufort,41610
3,Beluga whale,Eastern Chukchi Sea,3710
4,Beluga whale,Norton Sound++,7367
5,Beluga whale,Bristol Bay,1555
6,Beluga Whale,Cook Inlet++,1251
7,Bowhead Whale,lolestern Arctic,8000
8,Cuvier's Beaked Whale,Alaska,-1
9,Dall's Porpoise,Alaska,83400


In [86]:
populationdata = data1995.population
plot(populationdata)

TypeError: 'NoneType' object is not iterable

R: Graph!

I: Do the results make sense? Can/should we trust the model? If so, what does this mean, and if not, why?